<a href="https://colab.research.google.com/github/faridkharis/skripsi/blob/main/1_Data_Preparation_(Scraping_and_Preprocessing).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping

In [19]:
!pip install google-play-scraper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
import pandas as pd 
from google_play_scraper import Sort, reviews_all

result = reviews_all(
    'com.oy.chat',
    sleep_milliseconds=0, # defaults to 0
    lang='id', # defaults to 'en'
    country='id', # defaults to 'us'
    sort=Sort.NEWEST, # defaults to Sort.MOST_RELEVANT
    filter_score_with=None # defaults to None(means all score)
)

df = pd.DataFrame(result)
df = df.drop(columns=['userName', 'userImage', 'thumbsUpCount', 'reviewCreatedVersion', 'replyContent', 'repliedAt'])

# df.columns = ['id', 'review', 'rating', 'tanggal']
df

,reviewId,content,score,at
0,65864af7-31b2-40f4-990e-34a4ff028f6a,👍👍👍👍,5,2023-02-25 10:44:02
1,38af8bc9-4d0c-4876-bffa-1594a0f9056e,Aplikasi penipu hanya untuk pinjaman online sj...,1,2023-02-23 14:14:12
2,ae34f40a-8184-4e04-8d9f-d66172cd4240,"Sampah, proses refund belum dilakukan dari bul...",1,2023-02-22 10:18:25
3,72afff6f-c64d-48c3-ba2f-7ed5801c96c0,"Ini maintenance 2 Minggu lebih , niat Gk sih ?",1,2023-02-15 14:09:18
4,3fef94d1-1a48-4673-b104-435c71e45a29,Verifikasi KTP aja bilangnya 1x24 jam masa lam...,1,2023-02-15 05:37:33
...,...,...,...,...
15969,b3fea17c-6a49-433b-8386-5cfb87a44815,Cukup baik,5,2017-09-29 04:27:02
15970,a8d2ed88-f990-40df-811f-93c5bb001a02,Lgsung 5,5,2017-09-28 02:25:33
15971,36ac9248-5074-4710-9448-3efa21b40ea5,"Keren nih aplikasinya,",5,2017-09-27 13:18:46
15972,4f4894db-9bbb-4982-b832-40f98f0927e0,Nice,5,2017-09-27 05:10:22


# Case Folding

In [23]:
df['content'] = df['content'].str.lower()

df.head()

,reviewId,content,score,at
0,65864af7-31b2-40f4-990e-34a4ff028f6a,👍👍👍👍,5,2023-02-25 10:44:02
1,38af8bc9-4d0c-4876-bffa-1594a0f9056e,aplikasi penipu hanya untuk pinjaman online sj...,1,2023-02-23 14:14:12
2,ae34f40a-8184-4e04-8d9f-d66172cd4240,"sampah, proses refund belum dilakukan dari bul...",1,2023-02-22 10:18:25
3,72afff6f-c64d-48c3-ba2f-7ed5801c96c0,"ini maintenance 2 minggu lebih , niat gk sih ?",1,2023-02-15 14:09:18
4,3fef94d1-1a48-4673-b104-435c71e45a29,verifikasi ktp aja bilangnya 1x24 jam masa lam...,1,2023-02-15 05:37:33


# Tokenizing

In [24]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
import nltk
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

In [25]:
# ------ Tokenizing ---------

nltk.download('punkt')

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
df['content'] = df['content'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return  re.sub(r"\d+", "", text)

df['content'] = df['content'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

df['content'] = df['content'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

df['content'] = df['content'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

df['content'] = df['content'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

df['content'] = df['content'].apply(remove_singl_char)

# NLTK word tokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

df['content'] = df['content'].apply(word_tokenize_wrapper)

df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,reviewId,content,score,at
0,65864af7-31b2-40f4-990e-34a4ff028f6a,[],5,2023-02-25 10:44:02
1,38af8bc9-4d0c-4876-bffa-1594a0f9056e,"[aplikasi, penipu, hanya, untuk, pinjaman, onl...",1,2023-02-23 14:14:12
2,ae34f40a-8184-4e04-8d9f-d66172cd4240,"[sampah, proses, refund, belum, dilakukan, dar...",1,2023-02-22 10:18:25
3,72afff6f-c64d-48c3-ba2f-7ed5801c96c0,"[ini, maintenance, minggu, lebih, niat, gk, sih]",1,2023-02-15 14:09:18
4,3fef94d1-1a48-4673-b104-435c71e45a29,"[verifikasi, ktp, aja, bilangnya, jam, masa, l...",1,2023-02-15 05:37:33
...,...,...,...,...
15969,b3fea17c-6a49-433b-8386-5cfb87a44815,"[cukup, baik]",5,2017-09-29 04:27:02
15970,a8d2ed88-f990-40df-811f-93c5bb001a02,[lgsung],5,2017-09-28 02:25:33
15971,36ac9248-5074-4710-9448-3efa21b40ea5,"[keren, nih, aplikasinya]",5,2017-09-27 13:18:46
15972,4f4894db-9bbb-4982-b832-40f98f0927e0,[nice],5,2017-09-27 05:10:22


# Normalization

In [26]:
from google.colab import drive
drive.mount('/content/drive/')

path = 'drive/MyDrive/Colab Notebooks/App Improvement Recommendation System/Data/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [27]:
normalizad_word = pd.read_excel(path + 'normalisasi.xlsx')

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

df['content'] = df['content'].apply(normalized_term)

df

,reviewId,content,score,at
0,65864af7-31b2-40f4-990e-34a4ff028f6a,[],5,2023-02-25 10:44:02
1,38af8bc9-4d0c-4876-bffa-1594a0f9056e,"[aplikasi, penipu, hanya, untuk, pinjaman, onl...",1,2023-02-23 14:14:12
2,ae34f40a-8184-4e04-8d9f-d66172cd4240,"[sampah, proses, refund, belum, dilakukan, dar...",1,2023-02-22 10:18:25
3,72afff6f-c64d-48c3-ba2f-7ed5801c96c0,"[ini, maintenance, minggu, lebih, niat, tidak,...",1,2023-02-15 14:09:18
4,3fef94d1-1a48-4673-b104-435c71e45a29,"[verifikasi, ktp, saja, bilangnya, jam, masa, ...",1,2023-02-15 05:37:33
...,...,...,...,...
15969,b3fea17c-6a49-433b-8386-5cfb87a44815,"[cukup, baik]",5,2017-09-29 04:27:02
15970,a8d2ed88-f990-40df-811f-93c5bb001a02,[lgsung],5,2017-09-28 02:25:33
15971,36ac9248-5074-4710-9448-3efa21b40ea5,"[keren, ini, aplikasinya]",5,2017-09-27 13:18:46
15972,4f4894db-9bbb-4982-b832-40f98f0927e0,[bagus],5,2017-09-27 05:10:22


In [28]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in df['content']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))

12163


In [29]:
for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    
    print(term,":" ,term_dict[term])

Output streaming akan dipotong hingga 5000 baris terakhir.
mencegah : cegah
penyebaran : sebar
virus : virus
nempel : nempel
loohjdi : loohjdi
lagiiii : lagiiii
fintech : fintech
ovowkwkwk : ovowkwkwk
panen : panen
berbasis : bas
translator : translator
menabung : tabung
bankmau : bankmau
nyobaa : nyobaa
bagusan : bagus
soo : soo
farrr : farrr
inovasiinovasi : inovasiinovasi
terimakasuh : terimakasuh
terawasi : awas
variatif : variatif
integrasi : integrasi
lisensi : lisensi
gembar : gembar
gembor : gembor
sekalisekali : sekalisekali
kutunggu : tunggu
telatnya : telat
sebanding : banding
kupake : kupake
confidential : confidential
orangorang : orangorang
barunya : baru
baguuusssss : baguuusssss
much : much
bingit : bingit
gasiasia : gasiasia
perangkat : perangkat
kebeda : beda
komena : komena
suruhan : suruh
group : group
project : project
whats : whats
isia : isia
ramahtrimakasih : ramahtrimakasih
efesiensi : efesiensi
luvvv : luvvv
digunakannya : guna
baikk : baikk
dicairkan : cair
b

In [30]:
# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

df['content'] = df['content'].swifter.apply(get_stemmed_term)

print(df['content'])

Pandas Apply:   0%|          | 0/15974 [00:00<?, ?it/s]

0                                                       []
1        [aplikasi, tipu, hanya, untuk, pinjam, online,...
2        [sampah, proses, refund, belum, laku, dari, bu...
3        [ini, maintenance, minggu, lebih, niat, tidak,...
4        [verifikasi, ktp, saja, bilang, jam, masa, lam...
                               ...                        
15969                                        [cukup, baik]
15970                                             [lgsung]
15971                               [keren, ini, aplikasi]
15972                                              [bagus]
15973                                 [like, it, aplikasi]
Name: content, Length: 15974, dtype: object


# Stopword Removal

In [31]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
from nltk.corpus import stopwords

# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend([ 'indonesia', 'kulina', 'aplikasi', 'bobobox', 'oy', 'stockbit', 'payfazz', 'fazz', 'bukuwarung', 'buku', 'warung', 
                       'happyfresh', 'ipiring', 'nya', 'iya', 'coba', 'sih', 'bintang', 'terima', 'kasih', 'terimakasih', 'the', 'deh', 'in', 
                       'pagi', 'dong', 'moga', 'ya', 'banget', 'jaya', 'maju', 'bagus', 'suka', 'lumayan', 'cepat', 'tambah', 'tingkat', 'layan', 
                       'ribet', 'oke', 'mantap', 'bantu', 'banget', 'pakai', 'terima', 'kasih', 'keren', 'mudah', 'manfaat', 'kecewa', 'tolong',
                       'gampang', 'sukses', 'admin', 'susah', 'turun', 'naikin', 'auto', 'nomor', 'uninstall', 'hp', 'ganti', 'jelek', 'download',
                       'bangkrut', 'capek', 'melulu', 'ya', 'hilang', 'masuk', 'hapus', 'lancar', 'pokok', 'alhamdulillah', 'biar', 'minimal', 
                       'laku', 'aman', 'jaga', 'terimakasih', 'terima kasih', 'gagal', 'gratis', 'uang'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

df['content'] = df['content'].apply(stopwords_removal) 

In [33]:
df

,reviewId,content,score,at
0,65864af7-31b2-40f4-990e-34a4ff028f6a,[],5,2023-02-25 10:44:02
1,38af8bc9-4d0c-4876-bffa-1594a0f9056e,"[tipu, pinjam, online, sja, di jalan, accord]",1,2023-02-23 14:14:12
2,ae34f40a-8184-4e04-8d9f-d66172cd4240,"[sampah, proses, refund, desember, februari]",1,2023-02-22 10:18:25
3,72afff6f-c64d-48c3-ba2f-7ed5801c96c0,"[maintenance, minggu, niat]",1,2023-02-15 14:09:18
4,3fef94d1-1a48-4673-b104-435c71e45a29,"[verifikasi, ktp, bilang, jam, si, gila, prose...",1,2023-02-15 05:37:33
...,...,...,...,...
15969,b3fea17c-6a49-433b-8386-5cfb87a44815,[],5,2017-09-29 04:27:02
15970,a8d2ed88-f990-40df-811f-93c5bb001a02,[lgsung],5,2017-09-28 02:25:33
15971,36ac9248-5074-4710-9448-3efa21b40ea5,[],5,2017-09-27 13:18:46
15972,4f4894db-9bbb-4982-b832-40f98f0927e0,[],5,2017-09-27 05:10:22


# Save to Drive

In [34]:
import numpy as np

In [35]:
# df2 = df.replace(r'^\s*$', np.nan, regex=True)
df2 = df[df.astype(str)['content'] != '[]']
df2

,reviewId,content,score,at
1,38af8bc9-4d0c-4876-bffa-1594a0f9056e,"[tipu, pinjam, online, sja, di jalan, accord]",1,2023-02-23 14:14:12
2,ae34f40a-8184-4e04-8d9f-d66172cd4240,"[sampah, proses, refund, desember, februari]",1,2023-02-22 10:18:25
3,72afff6f-c64d-48c3-ba2f-7ed5801c96c0,"[maintenance, minggu, niat]",1,2023-02-15 14:09:18
4,3fef94d1-1a48-4673-b104-435c71e45a29,"[verifikasi, ktp, bilang, jam, si, gila, prose...",1,2023-02-15 05:37:33
5,ebc3ddaa-e20f-4a0b-ad97-3d742a109b98,"[bayar, via, bank, bank]",5,2023-02-10 01:02:20
...,...,...,...,...
15966,1d26639e-3db6-4141-b976-44cc0def63da,[so],5,2017-09-29 09:22:47
15967,a44fb20b-3bf4-4a60-92af-ee38f68e8556,"[kembang, tunggu, update, fitur, muas, install]",5,2017-09-29 06:00:03
15968,a3337d05-f5db-42d2-9690-d36d4274df5b,[perfect],5,2017-09-29 05:51:53
15970,a8d2ed88-f990-40df-811f-93c5bb001a02,[lgsung],5,2017-09-28 02:25:33


In [36]:
with open(path + 'data-ulasan-oy-preprocessed.csv', 'w') as f:
  df2.to_csv(f)